In [ ]:
import os
import pickle
import datetime
import pandas as pd
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input, Reshape, Permute
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D, Conv2D, DepthwiseConv2D, SeparableConv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt

# AU definitions
au_columns_name = {}
au_columns_name['AU01_r'] = 'Inner brow raiser, upper'
au_columns_name['AU02_r'] = 'Outer brow raiser, upper'
au_columns_name['AU04_r'] = 'Brow lowerer, upper'
au_columns_name['AU05_r'] = 'Upper lid raiser, upper'
au_columns_name['AU06_r'] = 'Cheekraiser, upper'
au_columns_name['AU07_r'] = 'Lid tightener, upper'
au_columns_name['AU09_r'] = 'Nose wrinkler, lower'
au_columns_name['AU10_r'] = 'Upper lip raiser, lower'
au_columns_name['AU12_r'] = 'Lip corner puller, lower'
au_columns_name['AU14_r'] = 'Dimpler, lower'
au_columns_name['AU15_r'] = 'Lip corner depressor, lower'
au_columns_name['AU17_r'] = 'Chin raiser, lower'
au_columns_name['AU20_r'] = 'Lipstretcher, lower'
au_columns_name['AU23_r'] = 'Lip tightener, lower'
au_columns_name['AU25_r'] = 'Lips part, lower'
au_columns_name['AU26_r'] = 'Jaw drop, lower'
au_columns_name['AU45_r'] = 'Blink, upper'

fps_df = pd.read_csv('../dataset/csv_labels/FPS_of_stutter_dataset.csv', index_col=0)
max_fps = max(fps_df["FPS"])

In [ ]:
subjects = ['942', '1131', '1196', '1214', '970', '971', '982']

X_array = []
Y_array = []

if len(subjects) > 1:
    for subject in subjects:
        with open('../dataset/pickled_datasets/X_array_balanced_upsampled_S1S2_'+ str(subject) +'.pkl','rb') as f: X_array.append(pickle.load(f))
        with open('../dataset/pickled_datasets/Y_array_balanced_upsampled_S1S2_'+ str(subject) +'.pkl','rb') as f: Y_array.append(pickle.load(f))
    X_array = np.vstack(X_array)
    Y_array = np.hstack(Y_array)
else:
    with open('../dataset/pickled_datasets/X_array_balanced_upsampled_S1S2_'+ str(subject) +'.pkl','rb') as f: X_array = pickle.load(f)
    with open('../dataset/pickled_datasets/Y_array_balanced_upsampled_S1S2_'+ str(subject) +'.pkl','rb') as f: Y_array = pickle.load(f)
        
print(X_array.shape, Y_array.shape)

In [ ]:
print("X_array.shape: ", X_array.shape)
Y_hist = np.histogram(Y_array)
Y_hist_sum = Y_hist[0][0] + Y_hist[0][-1]
print("Fluent Trials: {} ({:.2f}%), Stutter Trials: {} ({:.2f}%)".format(Y_hist[0][0], 100*(Y_hist[0][0]/Y_hist_sum), Y_hist[0][-1], 100*(Y_hist[0][-1]/Y_hist_sum)))

In [ ]:
# _StutterNet_C_S1S2. Functional API model.

def define_model():
    K.set_image_data_format('channels_first')

    clear_session()

    channels = 17
    timesteps = 87 # upsampled 58 fps

    inputs = Input(shape=(channels, timesteps))

    input_permute = Permute((1, 2), input_shape=(channels, timesteps))(inputs)
    input_reshape = Reshape((1, channels, timesteps))(input_permute)

    conv2d_1 = Conv2D(64, (1,17), activation='linear', input_shape=(channels, timesteps), padding='same')(input_reshape)
    conv2d_1_bn = BatchNormalization()(conv2d_1)

    conv2d_2DW = DepthwiseConv2D((17,1), use_bias=False, activation='linear', depth_multiplier=2, padding='valid', kernel_constraint=max_norm(1.))(conv2d_1_bn)
    conv2d_2DW_bn = BatchNormalization()(conv2d_2DW)
    conv2d_2DW_bn_act = Activation('elu')(conv2d_2DW_bn)

    conv2d_2DW_bn_act_avpool = AveragePooling2D((1,4))(conv2d_2DW_bn_act)
    conv2d_2DW_bn_act_avpool_dp = Dropout(rate=0.25)(conv2d_2DW_bn_act_avpool)

    conv2d_3Sep = SeparableConv2D(64, (1, 16), activation='linear', padding='same')(conv2d_2DW_bn_act_avpool_dp)
    conv2d_3Sep_bn = BatchNormalization()(conv2d_3Sep)
    conv2d_3Sep_bn_act = Activation('elu')(conv2d_3Sep_bn)

    conv2d_3Sep_bn_act_avgpool = AveragePooling2D((1,8))(conv2d_3Sep_bn_act)
    conv2d_3Sep_bn_act_avgpool_dp = Dropout(rate=0.25)(conv2d_3Sep_bn_act_avgpool)

    flatten_1 = Flatten()(conv2d_3Sep_bn_act_avgpool_dp)
    dense_1 = Dense(64, activation='elu', kernel_constraint=max_norm(0.5))(flatten_1)
    # dense_1_reshape = Reshape((64, 1))(dense_1)

    # lstm_1 = LSTM(64, input_shape=(64, 1))(dense_1_reshape)
    # lstm_1_act = Activation('elu')(lstm_1)

    predictions = Dense(1, activation='sigmoid', kernel_constraint=max_norm(0.25))(dense_1)

    model = Model(inputs=inputs, outputs=predictions)
    
    return model

def plot_history(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    
# model = define_model()
# model.summary()
# plot_model(model, to_file='StutterNet_C.png')


In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

X_array_, X_test, Y_array_, y_test = train_test_split(
    X_array, Y_array, test_size=0.3, random_state=seed)

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=15, min_lr=0.001)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

accuracies = []
precisions = []
recalls = []
f1s = []
count = 1

for train, test in kfold.split(X_array_, Y_array_):
    print("Processing Fold ", count)
    count += 1
    model = define_model()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_array[train], Y_array[train], 
                        epochs=100, batch_size=256, verbose=0, 
                        validation_data=(X_array[test], Y_array[test]),
                        shuffle=True,
                        callbacks=[reduce_lr])

    plot_history(history)

    test_output = model.evaluate(X_test, y_test, verbose=0)
    y_pred = model.predict(X_test)
    y_pred_binary = np.round(np.clip(y_pred, 0, 1)).flatten()
    precision = precision_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    print("Accuracy: {:.2f}%, Recall: {:.3f}, Precision: {:.3f}, F1: {:.3f}".format(test_output[1]*100, recall, precision, f1))
    
    accuracies.append(test_output[1]*100)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    
    if test_output[1] > 0.8:
        model_name = '_StutterNet_C_upscaled_S1S2_subject_' + str(subject) + '_'
        model_path = '../trained_models/' + str(datetime.date.today()) + model_name + '{:.3f}'.format(test_output[1])[-3:]
        print("Saving to: ", model_path + '.h5')
        model.save(model_path + '.h5')
        with open(model_path + '_history.pkl','wb') as f: pickle.dump(model.history.history, f)
        with open(model_path + '_params.pkl','wb') as f: pickle.dump(model.history.params, f)

    ##     with open(model_path + '_history.pkl','rb') as f: history2 = pickle.load(f)
    ##     with open(model_path + '_params.pkl','rb') as f: params2 = pickle.load(f)

print("Training Statistics")
metrics_df = pd.DataFrame.from_dict({'Accuracy': accuracies, 'F1': f1s, 'Recall': recalls, 'Precision': precisions})
print(metrics_df.describe())